## Merge the Defense_db and the Offense_db into a singular database

In [8]:
'''
This script merges the defense_db.csv and the offense_db.csv into a file named full_database.csv 
and adds additional lag components of the downloaded statistics to the dataframe for offense and defense
'''

# import packages
import pandas as pd

# read in the defense dataframe
defense = pd.read_csv('defense_db.csv')
defense.TO = defense.TO.fillna(0) # fill NaN's with 0's for the Turnover category (TO)
defense.location = defense.location.fillna('home') # set game location based on missing values for home/away
defense.dropna(inplace=True)
defense.reset_index(drop=True,inplace=True)

# date placeholder dataframe for date conversion
df1 = pd.DataFrame(columns = ['Date'], index=defense.index)
for i in range(len(defense)):
  df1.Date[i] = pd.to_datetime(defense.date[i])

# index by datetime, copy the 'd' column to the defense column and then drop 3 unused columns
defense.index = df1.Date
defense['Defense'] = defense.d
defense.drop(columns=['Unnamed: 0', 'date', 'd'], inplace = True)

# read in the offense dataframe
df = pd.read_csv('offense_db.csv')
df.index = pd.to_datetime(df.Date) #index by datetime
df.drop(columns=['Unnamed: 0', 'Date'], inplace=True) # drop unused columns

# unique nfl teams
nfl_teams = []
for x in defense.opp:
  if x not in nfl_teams:
    nfl_teams.append(x)
nfl_teams

# unique nfl team abbreviations
nfl_abbreviations = []
for x in df.Team:
  if x not in nfl_abbreviations:
    nfl_abbreviations.append(x)
nfl_abbreviations

# create a dataframe of the nfl teams and their unique teams 
# the nature of this is to make sure the location changes of teams such as the raiders, rams and chargers are properly captured
nfl = pd.DataFrame(columns = ['Team', 'Key'], index = range(len(nfl_teams)))
nfl.Team = nfl_teams
nfl.Key[0] = nfl_abbreviations[nfl_abbreviations.index('DAL')]
nfl.Key[1] = nfl_abbreviations[nfl_abbreviations.index('PHI')]
nfl.Key[2] = nfl_abbreviations[nfl_abbreviations.index('BUF')]
nfl.Key[3] = nfl_abbreviations[nfl_abbreviations.index('NYG')]
nfl.Key[4] = nfl_abbreviations[nfl_abbreviations.index('WAS')]
nfl.Key[5] = nfl_abbreviations[nfl_abbreviations.index('SFO')]
nfl.Key[6] = nfl_abbreviations[nfl_abbreviations.index('CHI')]
nfl.Key[7] = nfl_abbreviations[nfl_abbreviations.index('MIA')]
nfl.Key[8] = nfl_abbreviations[nfl_abbreviations.index('SEA')]
nfl.Key[9] = nfl_abbreviations[nfl_abbreviations.index('TAM')]
nfl.Key[10] = nfl_abbreviations[nfl_abbreviations.index('ARI')]
nfl.Key[11] = nfl_abbreviations[nfl_abbreviations.index('NWE')]
nfl.Key[12] = nfl_abbreviations[nfl_abbreviations.index('NYJ')]
nfl.Key[13] = nfl_abbreviations[nfl_abbreviations.index('GNB')]
nfl.Key[14] = nfl_abbreviations[nfl_abbreviations.index('CLE')]
nfl.Key[15] = nfl_abbreviations[nfl_abbreviations.index('HOU')]
nfl.Key[16] = nfl_abbreviations[nfl_abbreviations.index('KAN')]
nfl.Key[17] = nfl_abbreviations[nfl_abbreviations.index('CIN')]
nfl.Key[18] = nfl_abbreviations[nfl_abbreviations.index('PIT')]
nfl.Key[19] = nfl_abbreviations[nfl_abbreviations.index('IND')]
nfl.Key[20] = nfl_abbreviations[nfl_abbreviations.index('TEN')]
nfl.Key[21] = nfl_abbreviations[nfl_abbreviations.index('JAX')]
nfl.Key[22] = nfl_abbreviations[nfl_abbreviations.index('DEN')]
nfl.Key[23] = nfl_abbreviations[nfl_abbreviations.index('BAL')]
nfl.Key[24] = nfl_abbreviations[nfl_abbreviations.index('LAR')]
nfl.Key[25] = nfl_abbreviations[nfl_abbreviations.index('DET')]
nfl.Key[26] = nfl_abbreviations[nfl_abbreviations.index('CAR')]
nfl.Key[27] = nfl_abbreviations[nfl_abbreviations.index('LVR')]
nfl.Key[28] = nfl_abbreviations[nfl_abbreviations.index('LAC')]
nfl.Key[29] = nfl_abbreviations[nfl_abbreviations.index('ATL')]
nfl.Key[30] = nfl_abbreviations[nfl_abbreviations.index('MIN')]
nfl.Key[31] = nfl_abbreviations[nfl_abbreviations.index('NOR')]
nfl.Key[32] = nfl_abbreviations[nfl_abbreviations.index('WAS')]
nfl.Key[33] = nfl_abbreviations[nfl_abbreviations.index('LVR')]
nfl.Key[34] = nfl_abbreviations[nfl_abbreviations.index('LAC')]
nfl.Key[35] = nfl_abbreviations[nfl_abbreviations.index('LAR')]
nfl.index = nfl.Key # index by the Key
#nfl.drop(columns=['Key'],inplace=True)

# set up a key identifier to merge the defense dataframe onto the offense dataframe
# first set up a key for the opponents of defenses
opp_keys = []
for i in range(len(defense)):
  for j in range(len(nfl.Team)):
    if defense.opp[i] == nfl.Team[j]:
      opp_keys.append(nfl.Key[j])

# then the opponent key and the date of the matchup are combined to create a unique key for each matchup
defense_teamkey = []
for i in range(len(defense)):
  defense_teamkey.append(str(opp_keys[i]) + ' ' + str(defense.index[i]))
defense["Team_Key"] = defense_teamkey
defense.dropna(inplace=True)

# add a date column to the offense dataframe
df["Date"] = df.index

# create a unique key for the offense consisting of their team name and the date of the matchup (this will let the defense merge onto the offense)
df_teamkey = []
for i in range(len(df)):
  df_teamkey.append(str(df.Team[i]) + ' ' + str(df.index[i]))
df["Team_Key"] = df_teamkey

# merge the defense dataframe onto the offense dataframe using the unique keys created above
nfl_database = pd.merge(df,defense, left_on = df.Team_Key, right_on = defense.Team_Key)
#nfl_database.index = nfl_database.Date
nfl_database.drop(columns = [ 'key_0', 'opp', 'Team_Key_y', 'Team_Key_x'], inplace=True) # drop unused columns

df = nfl_database # rename main df to be the nerged nfl_database

# sort values by date and reset index
df.sort_values('Date', ascending = False,inplace=True)
df.reset_index(drop=True,inplace=True)

# lists for 1, 2 and 3 lags of the offensive player fantasy points for the matchup
ffp_1 = []
ffp_2 = []
ffp_3 = []

# filter through each element in the merged dataframe and calculate the lags
for i in range(len(df)):
  player = df[df.Name == df.Name[i]]
  player['spot'] = player.index # track the spot
  player.reset_index(drop=True, inplace=True)
  player['FFP_lag1'] = player.FD_points.shift(-1) # 1 lag
  player['FFP_lag2']= player.FD_points.shift(-2) # 2 lags
  player['FFP_lag3'] = player.FD_points.shift(-3) # 3 lags
  
  # append lags to the empty lists
  for j in range(len(player)):
    if player['spot'][j] == df.index[i]:
        ffp_1.append(player['FFP_lag1'][j])
        ffp_2.append(player['FFP_lag2'][j])
        ffp_3.append(player['FFP_lag3'][j])
  
# add new columns to the nfl_dataframe to add the three lags of the offensive player to each matchup
df['FFP_lag1'] = ffp_1
df['FFP_lag2'] = ffp_2
df['FFP_lag3'] = ffp_3

# lists for 1, 2, and 3 lags of the defensive team total yards allowed, pass yards allowed and rush yards allowed
d1 = [] # lag 1, total yards
d2 = [] # lag 1, pass yards
d3 = [] # lag 1, rush yards
d12 = [] # lag 2, total yards 
d22 = [] # lag2, pass yards
d32 = [] # lag 2, rush yards
d13 = [] # lag3, total yards
d23 = [] # lag3, pass yards
d33 = [] # lag3, rush yards

# filter through each element in the list and calculate all of the lag components from the defense
for i in range(len(df)):
  player = df[df.Name == df.Name[i]]
  player['spot'] = player.index
  player.reset_index(drop=True, inplace=True)
  d = defense[defense['Defense']==df.Defense[i]]
  d.sort_values('Date', ascending = False,inplace=True)
  d['Tot_lag1'] = d.TotYd.shift(-1)
  d['Pass_lag1']= d.PassY.shift(-1)
  d['Rush_lag1'] = d.RushY.shift(-1)
  d['Tot_lag2'] = d.TotYd.shift(-2)
  d['Pass_lag2']= d.PassY.shift(-2)
  d['Rush_lag2'] = d.RushY.shift(-2)
  d['Tot_lag3'] = d.TotYd.shift(-3)
  d['Pass_lag3']= d.PassY.shift(-3)
  d['Rush_lag3'] = d.RushY.shift(-3)
  
  # append lags to the empty lists
  for j in range(len(d)):
    if str(d['Team_Key'][j]) == str(df.Team[i]) + ' ' + str(df.Date[i]):
        d1.append(d['Tot_lag1'][j])
        d2.append(d['Pass_lag1'][j])
        d3.append(d['Rush_lag1'][j])
        d12.append(d['Tot_lag2'][j])
        d22.append(d['Pass_lag2'][j])
        d32.append(d['Rush_lag2'][j])
        d13.append(d['Tot_lag3'][j])
        d23.append(d['Pass_lag3'][j])
        d33.append(d['Rush_lag3'][j])

# add new columns to the nfl_dataframe to add the nine lags of the defensive team to each matchup  
df['Tot_lag1'] = d1
df['Pass_lag1'] = d2
df['Russ_lag1'] = d3
df['Tot_lag2'] = d12
df['Pass_lag2'] = d22
df['Russ_lag2'] = d32
df['Tot_lag3'] = d13
df['Pass_lag3'] = d23
df['Russ_lag3'] = d33

# save the full_database to a CSV
df.to_csv('full_database.csv')



In [3]:
defense

,opp,first_downs,TotYd,PassY,RushY,TO,day,kickoff,location,Defense,Team_Key
date,,,,,,,,,,,
September 13 2020,Dallas Cowboys,24.0,380.0,244.0,136.0,0.0,Sun,8:20PM ET,home,Rams,DAL September 13 2020
September 20 2020,Philadelphia Eagles,25.0,363.0,242.0,121.0,3.0,Sun,1:00PM ET,@,Rams,PHI September 20 2020
September 27 2020,Buffalo Bills,23.0,375.0,274.0,101.0,2.0,Sun,1:00PM ET,@,Rams,BUF September 27 2020
October 4 2020,New York Giants,19.0,295.0,159.0,136.0,1.0,Sun,4:05PM ET,home,Rams,NYG October 4 2020
October 11 2020,Washington Football Team,10.0,108.0,70.0,38.0,0.0,Sun,1:00PM ET,@,Rams,WAS October 11 2020
...,...,...,...,...,...,...,...,...,...,...,...
December 11 2011,Minnesota Vikings,29.0,425.0,156.0,269.0,6.0,Sun,1:03PM ET,home,Lions,MIN December 11 2011
December 18 2011,Oakland Raiders,26.0,477.0,345.0,132.0,1.0,Sun,4:05PM ET,@,Lions,LVR December 18 2011
December 24 2011,San Diego Chargers,20.0,367.0,299.0,68.0,2.0,Sat,4:05PM ET,home,Lions,LAC December 24 2011
